In [166]:
import pandas as pd
import numpy as np
import numpy_financial as npf
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 4)

In [167]:
def cpr_smm(month: int, psa: int, seasoning: int, psa_parameters: list)-> tuple:

    if month + seasoning <= psa_parameters[1][0]:
        c = psa_parameters[0][1] + (psa_parameters[1][1] - psa_parameters[0][1])*(month + seasoning - 1)/(psa_parameters[1][0] - psa_parameters[0][0])
        
    else:
        c = psa_parameters[1][1] + (psa_parameters[2][1] - psa_parameters[1][1])*(month + seasoning - psa_parameters[1][0])/(psa_parameters[2][0] - psa_parameters[1][0])
    cpr = psa * c
    smm = 1 - (1-cpr)**(1/12)
    return cpr, smm


def mbs_passthrough(start_balance: float, mortgage_term: int, mortgage_rate: float, pass_through_rate: float, psa: int, seasoning: int, psa_parameters: list) -> pd.DataFrame:
    months = np.arange(1, mortgage_term+1, 1).tolist()
    
    data = []
    begin_balance = start_balance
    for m in months:
        month = m 
        cpr, smm = cpr_smm(month=m, psa=psa, seasoning=seasoning, psa_parameters=psa_parameters)
        sb = begin_balance
#         print(m, -mortgage_term+seasoning+m-1)
        mp = np.round((sb*(mortgage_rate/12))/(1-(1+mortgage_rate/12)**(-mortgage_term+seasoning+m-1)), 3)
        mhint = mortgage_rate * sb / 12
        ptint = pass_through_rate * sb /12
        sh_principal_repay = mp - mhint
#         print(sb - sh_principal_repay, smm, (sb - sh_principal_repay)*smm)
        prepayment = np.round((sb - sh_principal_repay)*smm, 3)
        
        total_prepayment = np.round(sh_principal_repay + prepayment, 3)
        eb = np.round(sb - total_prepayment, 3)
        data.append([m, cpr, smm, sb, mp, mhint, ptint, sh_principal_repay, prepayment, total_prepayment, eb])
        begin_balance = eb
#         print(sb, eb)
        
    df = pd.DataFrame(data=data, columns=['Month', 'CPR', 'SMM', 'BeginMonthlyBalance', 'MonthlyPayment', 'MonthlyInterestMH', 'MonthlyInterestPT', 'ScheduledPrincipalPayment', 'Prepayment', 'TotalPrincipalPayment', 'EndMortgageBalance'])    
    return df        
        
        

### Question 1
Background: In this question, monthly mortgage coupon rates should be calculated by simply dividing the annual rate by 12. You should also assume that all of the securities pay monthly. You should divide annual interest rates by 12 to get the corresponding monthly rate and assume monthly compounding when computing present values.

(Level-Payment Mortgages) Compute the monthly payment on a 30-year level payment mortgage assuming an annual mortgage rate of 5% and an initial mortgage principal of $400,000.

Submission Guideline: Give your answer rounded to two decimal places. For example, if you compute the answer to be $73.2367, submit 73.24.

In [168]:
n_in_years = 30
r = 0.05
mortgage = 400000

pmt = -1*npf.pmt(rate=r/12,nper=n_in_years*12,pv=mortgage)
print(f"The monthly payment for mortgage ${mortgage} for term {n_in_years} years is ${np.round(pmt,2)}")

The monthly payment for mortgage $400000 for term 30 years is $2147.29


### Question 2
Background: Assume the same conditions as the previous question.

(Mortgage Pass-Throughs) Consider a $400 million pass-through MBS that has just been created (so the 'seasoning' of the pass-through is equal to 0). The underlying pool of mortgages each has a maturity of 20 years and an annual mortgage coupon rate of 6\%. The pass-through rate of the mortgage pool is 5\%. Assuming a prepayment multiplier of 100 PSA, what is the total amount of interest paid to the pass-through investors?

Submission Guideline: Give your answer in millions rounded to two decimal places. For example, if you compute the answer to be $123,456,789,12, submit 123.46.

In [169]:
remaining_mortgage = 400
mortgage_rate = 0.06
pass_through_rate = 0.05
seasoning = 0
term_of_loan = 20*12
psa = 1
psa_parameters = [[1, 0.002], [30, 0.06], [term_of_loan, 0.06]]
initial_monthly_payment = np.round((remaining_mortgage*(mortgage_rate/12))/(1-(1+mortgage_rate/12)**(-term_of_loan+seasoning)), 3)
psa_parameters

[[1, 0.002], [30, 0.06], [240, 0.06]]

In [170]:
print(cpr_smm(month=28, psa=psa,seasoning=seasoning, psa_parameters=psa_parameters))


(0.055999999999999994, 0.004790912859462759)


In [171]:
df = mbs_passthrough(start_balance=remaining_mortgage, mortgage_term=term_of_loan, mortgage_rate=mortgage_rate, pass_through_rate=pass_through_rate, psa=psa, seasoning=seasoning, psa_parameters=psa_parameters)
df

,Month,CPR,SMM,BeginMonthlyBalance,MonthlyPayment,MonthlyInterestMH,MonthlyInterestPT,ScheduledPrincipalPayment,Prepayment,TotalPrincipalPayment,EndMortgageBalance
0,1,0.002,0.0002,400.000,2.866,2.0000,1.6667,0.8660,0.067,0.933,399.067
1,2,0.004,0.0003,399.067,2.865,1.9953,1.6628,0.8697,0.133,1.003,398.064
2,3,0.006,0.0005,398.064,2.864,1.9903,1.6586,0.8737,0.199,1.073,396.991
3,4,0.008,0.0007,396.991,2.863,1.9850,1.6541,0.8780,0.265,1.143,395.848
4,5,0.010,0.0008,395.848,2.861,1.9792,1.6494,0.8818,0.331,1.213,394.635
5,6,0.012,0.0010,394.635,2.859,1.9732,1.6443,0.8858,0.396,1.282,393.353
6,7,0.014,0.0012,393.353,2.856,1.9668,1.6390,0.8892,0.461,1.350,392.003
7,8,0.016,0.0013,392.003,2.852,1.9600,1.6333,0.8920,0.525,1.417,390.586
8,9,0.018,0.0015,390.586,2.848,1.9529,1.6274,0.8951,0.589,1.484,389.102
9,10,0.020,0.0017,389.102,2.844,1.9455,1.6213,0.8985,0.653,1.551,387.551


In [172]:
print(f"The total amount of Interest Paid to PassThrough Investor is ${np.round(df['MonthlyInterestPT'].sum(), 2)}")

The total amount of Interest Paid to PassThrough Investor is $171.17


### Question 3 
Background: Assume the same conditions as the previous question.

(Mortgage-Pass Throughs) Referring to the same mortgage pass-through of the previous question, what is the total amount of the prepayments?

Submission Guideline: Give your answer in millions rounded to two decimal places. For example, if you compute the answer to be $123,456,789,12, submit 123.46.

In [173]:
print(f"The total prepayment is ${np.round(df['Prepayment'].sum(), 2)}")

The total prepayment is $181.09


### Question 4
Background: Assume the same conditions as the previous question.

(Mortgage-Pass Throughs) Referring to the same mortgage pass-through of the previous question, what is the total amount of the prepayments if the rate of prepayments increases to 200 PSA? 

Submission Guideline: Give your answer in millions rounded to two decimal places. For example, if you compute the answer to be $123,456,789,12, submit 123.46.

In [174]:
new_psa = 2
df_200psa = mbs_passthrough(start_balance=remaining_mortgage, mortgage_term=term_of_loan, mortgage_rate=mortgage_rate, pass_through_rate=pass_through_rate, psa=new_psa, seasoning=seasoning, psa_parameters=psa_parameters)
df_200psa

,Month,CPR,SMM,BeginMonthlyBalance,MonthlyPayment,MonthlyInterestMH,MonthlyInterestPT,ScheduledPrincipalPayment,Prepayment,TotalPrincipalPayment,EndMortgageBalance
0,1,0.004,0.0003,400.000,2.866,2.0000,1.6667,0.8660,0.133,0.999,399.001
1,2,0.008,0.0007,399.001,2.865,1.9950,1.6625,0.8700,0.266,1.136,397.865
2,3,0.012,0.0010,397.865,2.863,1.9893,1.6578,0.8737,0.399,1.273,396.592
3,4,0.016,0.0013,396.592,2.860,1.9830,1.6525,0.8770,0.532,1.409,395.183
4,5,0.020,0.0017,395.183,2.856,1.9759,1.6466,0.8801,0.663,1.543,393.640
5,6,0.024,0.0020,393.640,2.851,1.9682,1.6402,0.8828,0.794,1.677,391.963
6,7,0.028,0.0024,391.963,2.846,1.9598,1.6332,0.8862,0.924,1.810,390.153
7,8,0.032,0.0027,390.153,2.839,1.9508,1.6256,0.8882,1.054,1.942,388.211
8,9,0.036,0.0031,388.211,2.831,1.9411,1.6175,0.8899,1.182,2.072,386.139
9,10,0.040,0.0034,386.139,2.823,1.9307,1.6089,0.8923,1.308,2.200,383.939


In [175]:
print(f"The total prepayment if psa is {new_psa} is ${np.round(df_200psa['Prepayment'].sum(), 2)}")

The total prepayment if psa is 2 is $268.15
